In [1]:
import matplotlib as plt
import pandas as pd
import numpy as np
import bs4
import re
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
keyWord = 'data science'
keyWord = keyWord.replace(' ', '%20')
urlorig='https://www.headhunter.com/jobs?Keywords='+keyWord+'&Radius=150&PageNumber=1&OrderBy=Relevance'
urlorig

'https://www.headhunter.com/jobs?Keywords=data%20science&Radius=150&PageNumber=1&OrderBy=Relevance'

In [63]:
def get_pages(soup_param, res_list):
    pages = soup_param.findAll("a", class_ = "searchnext")
    if len(pages) > 0:
        link = pages[0]
        link_esc = link["href"].replace(' ', '%20')
        Jb_page = uReq("https://www.headhunter.com" + link_esc)
        Jb_page_html = Jb_page.read()
        Jb_page_soup = soup(Jb_page_html, 'html.parser')
        Jb_page.close()
        res_list.append(Jb_page_soup)
        get_pages(Jb_page_soup, res_list)


my_url = urlorig
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,'html.parser')

jb_all = []


links_all_jobs = page_soup.findAll("a", class_ = "see_all_jobs_link")
for link in links_all_jobs:
    Jb_all = uReq("https://www.headhunter.com" + link["href"])
    Jb_all_html = Jb_all.read()
    Jb_all.close()
    Jb_all_soup = soup(Jb_all_html, "html.parser")
    jb_all.append(Jb_all_soup)
    
all_pages=[]
for item in jb_all:
    get_pages(item, all_pages)

jb_all.extend(all_pages)


3
7


In [70]:
#single vacancy page
my_url1 = "https://www.headhunter.com/jobs/Senior-Data-Scientist/J3N61M605D08SG4PV0C"
uClient1 = uReq(my_url1)
page_html1 = uClient1.read()
uClient1.close()
page_soup1 = soup(page_html1,'html.parser')

print(page_soup1)



<!DOCTYPE html>

<html>
<head>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"13840fb14e","applicationID":"147043153","transactionName":"Jg0IFxRbWVhTEEkBBjoLCBcKG19bVBFJBgERAw8P","queueTime":0,"applicationTime":1351,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"13840fb14e",applicationID:"147043153"};window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof wi

In [164]:
# content = page_soup1.findAll("div", class_="content")
# print(len(content))
# for i in content:
#     print(i.text)
#     refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
#     print(refID.group()[8:])

job_text = page_soup1.findAll("div", id="description-container-standard")
item = job_text[0]
contents = item.select("div.content")
# print(contents)
descr=''
for content in contents:
    refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
    if refID != None:
        print(refID.group()[8:])
    
    descr+= content.text
    
    for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
        if i.text == 'Location ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Employment Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Pay Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Pay Rate ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Store Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Other Compensation: ':
            print(i.next_sibling.string.next_sibling.text)
# print(descr)

04030-0011057821
US-TX-Fort Worth
Full-Time
Year
N/A
Engineering,  Information Technology,  Design
DOE


In [64]:
titles = []
company_names = []
location_names = []
compensation = []
job_descriptions = []
job_ids = []
skills = {}
skills['Python'] = []
skills['R '] = []
skills['Matlab'] = []
skills['Excel'] = []
skills['SAS'] = []
skills['SQL'] = []
skills['SPSS'] = []
skills['Hadoop'] = []
skills['Spark'] = []

# extract job_title
jobs = page_soup.findAll("div", class_=re.compile("searchresult clearfix"))
for job in jobs:
    try:
        titles.append(job["title"])
    except:
        titles.append("NA")

# print(titles)

# extract company_name
companies = page_soup.findAll("span", class_="resultcompany_mobile clearfix" )
for company in companies:
    try:
        company_names.append(company.text.strip())
    except:
        company_names.append("NA")
        
# print(company_names)

# extract location       
locations = page_soup.findAll("span", class_="result-right-one")
for location in locations:
    try:
        location_names.append(location.text)
    except:
        location_names.append("NA")
        
# print(location_names)

# extract compensation       
payments = page_soup.findAll("span", class_="result-right-two non-mobile")
for payment in payments:
    try:
        compensation.append(payment.text)
    except:
        compensation.append("NA")
        
# print(compensation)

#extract job_id and full description
links = page_soup.findAll("a", class_ = "result-left")
for link in links:
    Jb = uReq("https://www.headhunter.com" + link["href"])
    Jb_html = Jb.read()
    Jb.close()
    Jb_soup = soup(Jb_html, "html.parser")
    try:
        content = Jb_soup.find("div", class_="content")
        refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
        job_ids.append(refID.group()[8:])
    except:
        job_ids.append("NA")    
        
    try:
        job_descriptions.append(str(re.sub('(\n|\r)', '',content.text)))
    except:
        job_descriptions.append("NA")
        
# print(job_descriptions)

#extract all jobsfor company with many vacancies
for item in jb_all:
    
    # extract job_title
    jobs = item.findAll("div", class_=re.compile("searchresult clearfix"))
    for job in jobs:
        try:
            titles.append(job["title"])
        except:
            titles.append("NA")
            
    # extract company_name
    companies = item.findAll("span", class_="resultcompany_mobile clearfix" )
    for company in companies:
        try:
            company_names.append(company.text.strip())
        except:
            company_names.append("NA")

    # extract location       
    locations = item.findAll("span", class_="result-right-one")
    for location in locations:
        try:
            location_names.append(location.text)
        except:
            location_names.append("NA")

    # extract compensation       
    payments = item.findAll("span", class_="result-right-two non-mobile")
    for payment in payments:
        try:
            compensation.append(payment.text)
        except:
            compensation.append("NA")

    #extract job_id and full description
    links = item.findAll("a", class_ = "result-left")
    for link in links:
        Jb = uReq("https://www.headhunter.com" + link["href"])
        Jb_html = Jb.read()
        Jb.close()
        Jb_soup = soup(Jb_html, "html.parser")
        try:
            content = Jb_soup.find("div", class_="content")
            refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
            job_ids.append(refID.group()[8:])
        except:
            job_ids.append("NA")    

        try:
            job_descriptions.append(str(re.sub('(\n|\r)', '',content.text)))
        except:
            job_descriptions.append("NA")

In [65]:
d = {'Job_Title':titles,'Company_Name':company_names, 'Location':location_names, 'Compensation': compensation, 'Job_ID':job_ids,'Job_Description':job_descriptions}
df1 = pd.DataFrame(d)
df2 = pd.DataFrame(skills)
frames = [df1, df2]
results = pd.concat(frames, axis = 1)
results.head()

,Job_Title,Company_Name,Location,Compensation,Job_ID,Job_Description,Python,R,Matlab,Excel,SAS,SQL,SPSS,Hadoop,Spark
0,Senior-Data-Scientist,Robert Half Technology,TX - Fort Worth,N/A,04030-0011057821,Ref ID: 04030-0011057821Classification: Data S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Senior-Data-Scientist,Robert Half Technology,NJ - Jersey City,$140k - $160k/year,02710-0011220068,Ref ID: 02710-0011220068Classification: Data S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Manager-of-IT-Audit-and-Data-Analytics,Robert Half Technology,GA - Atlanta,$120k - $130k/year,00900-0011225244,Ref ID: 00900-0011225244Classification: Sr. IT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Senior-Data-Engineer,Robert Half Technology,TX - Austin,$115k - $130k/year,04160-0010956620,Ref ID: 04160-0010956620Classification: Big Da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data-Analytics-Manager,"TEKsystems, Inc",GA - Atlanta,N/A,NA,***** CANDIDATES MUST BE ABLE TO WORK W2 ****...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
results.tail()

,Job_Title,Company_Name,Location,Compensation,Job_ID,Job_Description,Python,R,Matlab,Excel,SAS,SQL,SPSS,Hadoop,Spark
151,Sr--Engineer-Industrial,Robert Half Technology,TN - Brentwood,$125k - $135k/year,03930-0011158081,Ref ID: 03930-0011158081Classification: Site E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,Sr--PHP-Developer,Robert Half Technology,GA - Atlanta,$100k - $120k/year,00900-9502269473,Ref ID: 00900-9502269473Classification: Sr. PH...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Sr--C--PHP-Engineer,Robert Half Technology,FL - Miami,$100k - $115k/year,01020-9502287410,Ref ID: 01020-9502287410Classification: Sr. So...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,Sr--C--PHP-Engineer,Robert Half Technology,FL - Miami,$100k - $115k/year,01020-9502298939,Ref ID: 01020-9502298939Classification: Sr. So...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,Sr--Business-Intelligence-Analyst,Robert Half Technology,NJ - Bridgewater Township,N/A,02720-0011196585,Ref ID: 02720-0011196585Classification: Busine...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
results.to_csv('Headhunters_job_vacancies_raw.csv')